# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e compramos e vendemos commodities:
- Soja, Milho, Trigo, Petróleo, etc.

Precisamos pegar na internet, de forma automática, a cotação de todas as commodites e ver se ela está abaixo do nosso preço ideal de compra. Se tiver, precisamos marcar como uma ação de compra para a equipe de operações.

Base de Dados: https://drive.google.com/drive/folders/1O9aE_Hen5smf_a6TsbVF6DM0ms4Ykj6s?usp=share_link

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [2]:
# firefox -> geckodriver
# chrome -> chromedriver

# Passo a Passo
# Passo 1: Abrir o navegador e acessar o Google
from selenium import webdriver
navegador = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver")
# navegador.get("https://www.google.com/")

/tmp/ipykernel_43648/1695333717.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  navegador = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver")


Comandos usados para instalação das ferramentas no Ubuntu:

* sudo pip install selenium
* sudo apt-get install chromium-chromedriver
* sudo pip install openpyxl

In [3]:
# Passo 2: Importar a base de dados
import pandas as pd

tabela = pd.read_excel("Material/commodities.xlsx")
display(tabela)

,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,NaN,NaN
1,Soja,163.59,NaN,NaN
2,Boi,282.20,NaN,NaN
3,Petróleo,424.37,NaN,NaN
4,Algodão,497.76,NaN,NaN
5,Açúcar,136.23,NaN,NaN
6,Café,1092.87,NaN,NaN
7,Ouro,321.77,NaN,NaN
8,Tilápia,9.05,NaN,NaN


In [4]:
import unicodedata

links = []
produtos = []
precos = []

# Passo 3: Percorrer cada produto da base de dados
for linha in tabela.index:
    produto = tabela.loc[linha, "Produto"]
    
    # Substituir os caracteres especiais já mapeados
    # produto = produto.replace("ó", "o").replace("ã", "a").replace("á", "a").replace("ç", "c").replace("ú", "u").replace("é", "e")

    # Substituir qualquer caracter especial - normalização para ascii e ignorando erros, depois decode para transformar de bytes para string
    produto = unicodedata.normalize("NFKD", produto).encode("ascii", "ignore").decode("utf-8")
    

    # Passo 4: Pesquisar o preço de cada produto
    link = f"https://www.melhorcambio.com/{produto}-hoje"
    navegador.get(link)


    # Passo 5: Atualizar o preço na base de dados
    preco = navegador.find_element('xpath', '//*[@id="comercial"]').get_attribute('value') # Para copiar o xpath do elemento ir em inspecionar do navegador
    
    # Trocar a vígula por ponto e parsear o número para float
    preco = preco.replace(".", "").replace(",", ".")
    tabela.loc[linha, "Preço Atual"] = float(preco) # localizar posição por linha e coluna e preencher o preço

    # Listas de ilustração
    produtos.append(produto)
    precos.append(preco)
    links.append(link)


In [ ]:
# Outros métodos:
#
# .click() -> clicar no elemento
# .send_keys("texto") -> escrever no elemento
# .get_attribute() -> pegar um valor do elemento

In [5]:
# Conferir logs
print(produtos)
print(precos)
print(links)

['Milho', 'Soja', 'Boi', 'Petroleo', 'Algodao', 'Acucar', 'Cafe', 'Ouro', 'Tilapia']
['56.51', '132.56', '267.30', '377.18', '395.63', '148.83', '1043.39', '313.08', '9.64']
['https://www.melhorcambio.com/Milho-hoje', 'https://www.melhorcambio.com/Soja-hoje', 'https://www.melhorcambio.com/Boi-hoje', 'https://www.melhorcambio.com/Petroleo-hoje', 'https://www.melhorcambio.com/Algodao-hoje', 'https://www.melhorcambio.com/Acucar-hoje', 'https://www.melhorcambio.com/Cafe-hoje', 'https://www.melhorcambio.com/Ouro-hoje', 'https://www.melhorcambio.com/Tilapia-hoje']


In [6]:
# Conferir tabela
display(tabela)

,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,56.51,NaN
1,Soja,163.59,132.56,NaN
2,Boi,282.20,267.30,NaN
3,Petróleo,424.37,377.18,NaN
4,Algodão,497.76,395.63,NaN
5,Açúcar,136.23,148.83,NaN
6,Café,1092.87,1043.39,NaN
7,Ouro,321.77,313.08,NaN
8,Tilápia,9.05,9.64,NaN


In [7]:
# Passo 6: Decidir quais produtos comprar

# Preencher a coluna de comprar se o preço atual for menor que o preço ideal
tabela["Comprar"] = tabela["Preço Atual"] < tabela["Preço Ideal"]  # Compara ambas as colunas inteiras de uma vez, linha por linha
display(tabela)

,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,56.51,True
1,Soja,163.59,132.56,True
2,Boi,282.20,267.30,True
3,Petróleo,424.37,377.18,True
4,Algodão,497.76,395.63,True
5,Açúcar,136.23,148.83,False
6,Café,1092.87,1043.39,True
7,Ouro,321.77,313.08,True
8,Tilápia,9.05,9.64,False


In [8]:
# Passo 7: Exportar a base atualizada para um excel
tabela.to_excel("Material/commodities_atualizado.xlsx", index=False) # Retirar o index

# Se fosse para usar o mesmo arquivo da base original é só usar o mesmo nome, o original vai ser substituído

In [9]:
# Fechar o navegador após finalização
navegador.quit()